# Modelo simplificado de un enlace químico
Hasta ahora hemos explorado el efecto que diferentes potenciales tienen 
sobre las funciones de onda y los niveles de energía asociados. Para concluir, 
vamos a utilizar lo que hemos aprendido en el caso del pozo de potencial para
hacer un modelo minimalista de un enlace químico. En este modelo, combinamos 
dos pozos definidos en nuestra coordenada y resolvemos numéricamente la ecuación
de Schrödinger. 

In [ ]:
import numpy as np
import scipy.linalg as spla
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
# atomic units
hbar = 1.0
m = 1.0
# set precision of numerical approximation
steps = 2000

A continuación definimos la profundidad de nuestro pozo de potenciail y los límites en el eje $x$ para nuestro cálculo.

In [ ]:
D = 100.0
A = 2.0
W = A/2

# create x-vector from -A to A
xvec = np.linspace(-A, A, steps)
# get step size
h = xvec[1] - xvec[0]

In [ ]:
def well_potential(W, x, x0, D):
    pot = -D*(np.sign(x - x0 +W) - np.sign(x - x0 -W))
    return pot


U = well_potential(W, xvec, 1, D) + well_potential(W, xvec, -1, D)

A continuación calculamos una representación matricial de la Laplaciana usando el método de las diferencias finitas:

In [ ]:
# create Laplacian via 3 point finite-difference method
Laplacian = (-2.0*np.diag(np.ones(steps)) + \
             np.diag(np.ones(steps-1),1) + \
             np.diag(np.ones(steps-1),-1))/(float)(h**2)

In [ ]:
# create the Hamiltonian
Hamiltonian = np.zeros((steps,steps))
[i,j] = np.indices(Hamiltonian.shape)
Hamiltonian[i==j] = U
Hamiltonian += (-0.5)*((hbar**2)/m)*Laplacian

Para obtener las funciones propias (eigenvectors) y las energías (eigenvalues) usamos una función eigh de Scipy.

In [ ]:
def diagonalize_hamiltonian(Hamiltonian):
    return spla.eigh(Hamiltonian)

In [ ]:
evals, evecs= diagonalize_hamiltonian(Hamiltonian)

In [ ]:
print (" lowest bound state energies:")
for i in range(6):
    print ('E(%g) = %.2f'%(i+1,evals[i]))

In [ ]:
fig, ax = plt.subplots(figsize=(6,8))
for i, v in enumerate(evecs.transpose()[:6]):
    #V_new, ScaleFactor = infinite_well_plot_scaling(evals, evecs, xvec, W)
    color = mpl.cm.jet_r((i)/(float)(6-1),1)
    ax.plot(xvec, 100*v + evals[i], c=color)
    ax.axhline(evals[i], c=color, ls='--')
ax.plot(xvec, U, c='gray', lw=5)
ax.set_xlim(xvec[0], xvec[-1])
#x.set_ylim(-evals[0], evals[6])
ax.set_xlabel('L', fontsize=14)
# set y label
ax.set_ylabel('Energy / (a.u.)', fontsize=14)